# How to use WEAC v2.0

### About the project
---
WEAC implements closed-form analytical models for the [mechanical analysis of dry-snow slabs on compliant weak layers](https://doi.org/10.5194/tc-14-115-2020), the [prediction of anticrack onset](https://doi.org/10.5194/tc-14-131-2020), and, in particular, allwos for stratified snow covers. The model covers propagation saw tests (a), and uncracked (b) or cracked (c) skier-loaded buried weak layers.

<img style="float: left; margin: 0 0 15px 0;" img src="../img/bc.png" alt="bc" width="400"/>

Please refer to the companion papers for model derivations, illustrations, dimensions, material properties, and kinematics:

- Rosendahl, P. L., & Weißgraeber, P. (2020). Modeling snow slab avalanches caused by weak-layer failure – Part 1: Slabs on compliant and collapsible weak layers. The Cryosphere, 14(1), 115–130. https://doi.org/10.5194/tc-14-115-2020
- Rosendahl, P. L., & Weißgraeber, P. (2020). Modeling snow slab avalanches caused by weak-layer failure – Part 2: Coupled mixed-mode criterion for skier-triggered anticracks. The Cryosphere, 14(1), 131–145. https://doi.org/10.5194/tc-14-131-2020

Completely rewritten in [Python](https://www.python.org) and built with [Atom](https://atom.io), [GitKraken](https://www.gitkraken.com), and [Jupyter](https://jupyter.org). Looking for the [MATLAB](https://www.mathworks.com/products/matlab.html) implementation? Check out [release v1.0](https://github.com/2phi/weac/releases/tag/v1.0.0).

### Installation
---
Install globally using the `pip` Package Installer for Python
```sh
pip install weac
```
or clone the repo
```sh
git clone https://github.com/2phi/weac
```
for local use.

Needs
- [Numpy](https://numpy.org/) for matrix operations
- [Scipy](https://www.scipy.org/) for solving optimization problems
- [Pandas](https://pandas.pydata.org/) for data handling
- [Matplotlib](https://matplotlib.org/) for plotting
- [Timeit](https://docs.python.org/3/library/timeit.html) for timing

### License
---
Copyright 2phi GbR, 2021.

We currently do not offer an open source license. Please contact us for private licensing options.

### Contact
---
E-mail: mail@2phi.de · Web: https://2phi.de · Project Link: [https://github.com/2phi/weac](https://github.com/2phi/weac)

# Usage
---

### Preamble

In [3]:
# === NOTEBOOK SETUP ========================================================

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# === SYSTEM PATHS ==========================================================

# REMOVE AFTER PIP INSTALLATION
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# === LOAD MODULES ==========================================================

# Weak layer anticrack nucleation model
import weac

# Prepare output
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150

### Snow profile database

<img src="../img/profiles.png" alt="profiles" width="300"/>

### Create model instances

In [5]:
# Propagation saw test with homogeneous snow slab
pst = weac.Layered(system='pst'x)

# One skier on slab with profile B
skier_on_B = weac.Layered(
    system='skier',
    mat='profile.B')

# Multiple skiers on slab with profile C
skiers_on_C = weac.Layered(
    system='skiers',
    mat='profile.C')